In [1]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt

# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K

from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from imutils import paths
from keras.utils import to_categorical
import random
import cv2
import tensorflowjs as tfjs

AttributeError: module 'plaidml.op' has no attribute 'PoolDataFormat'

In [ ]:
# initialize the number of epochs to train for, initial learning rate,
# and batch size
EPOCHS = 75
INIT_LR = 1e-3
BS = 32
IMG_DIMS = (224, 224, 3)

MODEL_NAME = "multi-class"

data_set_dir = 'data/dataset/' + MODEL_NAME

In [ ]:
def split_train_test_set(directory):
    
    classifiers = next(os.walk(data_set_dir))[1]
    
    print("[INFO] loading images from {}".format(directory))
    # initialize the data and labels
    data = []
    labels = []
    
    # grab the image paths and randomly shuffle them
    image_paths = sorted(list(paths.list_images(directory)))
    random.seed(42)
    random.shuffle(image_paths)
    
    # loop over the input images
    for image_path in image_paths:
        # load the image, pre-process it, and store it in the data list
        image = cv2.imread(image_path)
        image = cv2.resize(image, (IMG_DIMS[1], IMG_DIMS[0]))
        image = img_to_array(image)
        data.append(image)
 
        # extract the class label from the image path and update the
        # labels list        
        l = label = image_path.split(os.path.sep)[-2].split("-")
        labels.append(l)
        
    # scale the raw pixel intensities to the range [0, 1]
    data = np.array(data, dtype="float") / 255.0
    labels = np.array(labels)
    
    print("[INFO] Labels")
    print(labels)
    
    mlb = MultiLabelBinarizer()
    labels = mlb.fit_transform(labels)
    
    # loop over each of the possible class labels and show them
    print("[INFO] Classifiers")
    for (i, label) in enumerate(mlb.classes_):
        print("{}. {}".format(i + 1, label))
    
    classifiers = mlb.classes_
    
    print("[INFO] Splitting the Train and Test sets")
    # partition the data into training and testing splits using 75% of
    # the data for training and the remaining 25% for testing
    train_X, test_X, train_Y, test_Y = train_test_split(data,
                                                        labels,
                                                        test_size=0.2,
                                                        random_state=42)

    # convert the labels from integers to vectors
#     train_Y = to_categorical(train_Y, num_classes=NUM_OF_CLASSES)
#     test_Y = to_categorical(test_Y, num_classes=NUM_OF_CLASSES)

    return train_X, test_X, train_Y, test_Y, classifiers

In [ ]:
def show_model_performance_graph(model, epochs):
    # plot the training loss and accuracy
    plt.style.use("ggplot")
    plt.figure()
    N = epochs
    plt.plot(np.arange(0, N), 
             model.history["loss"], 
             label="train_loss")
    plt.plot(np.arange(0, N), 
             model.history["val_loss"], 
             label="val_loss")
    
    plt.plot(np.arange(0, N), 
             model.history["acc"], 
             label="train_acc")
    plt.plot(np.arange(0, N), 
             model.history["val_acc"], 
             label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.show()

In [ ]:
def build_model(num_of_classes):
    
    model = Sequential()
    inputShape = (height, width, depth) = IMG_DIMS
    chanDim = -1
 
    # if we are using "channels first", update the input shape
    # and channels dimension
    if K.image_data_format() == "channels_first":
        inputShape = (depth, height, width)
        chanDim = 1

    # CONV => RELU => POOL
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))
    
    # (CONV => RELU) * 2 => POOL
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    # (CONV => RELU) * 2 => POOL
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
        
    # first (and only) set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
 
    # use a *softmax* activation for single-label classification
    # and *sigmoid* activation for multi-label classification
    model.add(Dense(num_of_classes))
    model.add(Activation('softmax'))
    
    model.summary()
    
    return model

In [ ]:
def train_model(dataset_dir):
    train_X, test_X, train_Y, test_Y, classifiers = split_train_test_set(dataset_dir)
    
    print("[INFO] Augmenting image data...")
    # construct the image generator for data augmentation
    aug = ImageDataGenerator(rotation_range=25, 
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode="nearest")
    
    # initialize the model
    print("[INFO] compiling model...")
    opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
    
    model = build_model(len(classifiers))
                  
    model.compile(loss="binary_crossentropy",
                  optimizer=opt, 
                  metrics=["accuracy"])
    
    # train the network
    print("[INFO] training network...")
    H = model.fit_generator(
        aug.flow(train_X, train_Y, batch_size=BS),
        validation_data=(test_X, test_Y), 
        steps_per_epoch=len(train_X) // BS,
        epochs=EPOCHS, 
        verbose=1)
    # save the model to disk
    print("[INFO] serializing network...")
    model.save(MODEL_NAME + "_model.h5")
    print("[INFO] serializing label binarizer...")
    
    with open(MODEL_NAME + '_labels.json', 'w') as f:
        for classify in classifiers:
            f.write(classify + '\n')
    
    return H, model, classifiers

In [ ]:
h, model, classifiers = train_model(data_set_dir)

In [ ]:
show_model_performance_graph(h, EPOCHS)

In [ ]:
# load the image
image = cv2.imread('./test3.jpg')
orig = image.copy()

# pre-process the image for classification
image = cv2.resize(image, (IMG_DIMS[0], IMG_DIMS[1]))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

In [ ]:
predictions = model.predict(image)[0]

In [ ]:
for (label, p) in zip(classifiers, predictions):
    print("{}: {:.2}%".format(label, p * 100))

In [ ]:
tfjs.converters.save_keras_model(model, MODEL_NAME + "_tfjs_model")